# [1] Import Library

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


# [2] Kode

Note:
1. Ubah url sesuai dengan pages yang ingin discraping

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

url = "https://www.paulaschoice.com/ingredient-dictionary?csortb1=ingredientNotRated&csortd1=1&csortb2=ingredientRating&csortd2=2&csortb3=name&csortd3=1&start=18000&sz=2000"
driver.get(url)

header = ["name", "rating", "description"]
file_path = "ingredients_page10.csv"
with open(file_path, "w", newline='', encoding="utf-8") as f:
    pd.DataFrame(columns=header).to_csv(f, index=False)

WebDriverWait(driver, 40).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".IngredientList__Table-sc-12d8srs-1.dFEoGO"))
)

soup = BeautifulSoup(driver.page_source, "html.parser")
ingredient_rows = soup.select(".IngredientList__Table-sc-12d8srs-1.dFEoGO tr")

for row in ingredient_rows:
    try:
        name = row.select_one(".IngredientList__Name-sc-12d8srs-10").get_text(strip=True)
    except AttributeError:
        name = None

    try:
        rating = row.select_one(".ColoredIngredientRating__Rating-sc-8nrs1d-0").get_text(strip=True)
    except AttributeError:
        rating = "Not Rated"

    try:
        description = row.select_one(".IngredientList__Description-sc-12d8srs-11").get_text(" ", strip=True)
    except AttributeError:
        description = None

    df = pd.DataFrame([{
        "name": name,
        "rating": rating,
        "description": description
    }])
    df.to_csv(file_path, mode="a", header=False, index=False, encoding="utf-8")
    print(f"Data for ingredient '{name}' written to CSV.")

driver.quit()
